## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


### (Advanced Users) Installing additional dependencies

Here we show you how to install additional packages into this Deepnote project. We will use the package [gcsopt](https://pypi.org/project/gcsopt/) as an example. To install additional dependencies, you have two choices:
- **Recommended** (automatically installs your dependencies every time you boot the Deepnote machine):
  1. Add the dependency to the list of dependencies in `reqirements.txt` on the left.  
      - **Note**: Make sure to NOT remove the text that is already in the file. You should add your dependencies on new lines AFTER `manipulation=year.month.day`. For our example, the file contents will look like:
          ```
          --extra-index-url https://drake-packages.csail.mit.edu/whl/nightly
          manipulation==2025.9.23
          gcsopt
          ```
  2. Restart the Deepnote machine (or run the `Init` notebook)
- **Alternatively:** Run `!pip install gcsopt` within a cell in the Deepnote notebook. You will have to do this everytime you restart your Deepnote machine.

### (Advanced Users) Updating the `manipulation` version
We continually add new features to the `manipulation` package, which is automatically updated in Deepnote. However, after you first click "Duplicate", you will stop receiving these updates. Follow these steps to update your version to the latest version:
1. Find the latest version number for `manipulation` on the [PyPi package page](https://pypi.org/project/manipulation/#history).
2. Update the version number for `manipulation=year.month.day` in the `requirements.txt` file on the left
3. Restart the Deepnote machine (or run the `Init` notebook)

The changes will persist every time you start your Deepnote machine.

Good luck!

---


## Placeholder code: iiwa Simulation

As a starting point, the code below sets up a simple simulation with a single iiwa, visualized in Meshcat.

### Setup and imports
Let us first get our imports out of the way:

In [2]:
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    BasicVector,
    LeafSystem,
    InverseKinematics,
    SnoptSolver,
    RotationMatrix,
    RigidTransform,
    Solve,
    LogVectorOutput,
    PiecewisePolynomial,
    TrajectorySource,
    ImageWriter,
    PixelType,
    JacobianWrtVariable,
    Context,
    MultibodyPlant,
    Integrator,
    ConstantVectorSource
)

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.geometry import Cylinder, Rgba

import matplotlib.pyplot as plt
import numpy as np
import pygame
import math
import os
import h5py
import numpy as np
import collections

pygame 2.6.1 (SDL 2.28.4, Python 3.13.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Users/aliciachen/Desktop/MIT/Fall25/6.4212/Manipulation-Final-Project/.venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


### Setting up the Scenario

In [4]:
import random
from pathlib import Path
DRAWER_CONFIGS = {
    "label53": {"handle_label": "handle5", "r_min": 0.28, "r_max": 0.31, "lambda": 0.00004, "kp_null": 1},
    "label62": {"handle_label": "handle3", "r_min": 0.4, "r_max": 0.45, "lambda": 0.004, "kp_null": 5},
    "label31": {"handle_label": "handle3", "r_min": 0.39, "r_max": 0.43,"lambda": 0.0004, "kp_null": 1.5},
    "label55": {"handle_label": "handle7", "r_min": 0.3, "r_max": 0.32,"lambda": 0.00001, "kp_null": 1}

}

def get_random_drawer_pose(
    drawer_name,  
    fov_deg=90, 
    face_noise_deg=30,
    cabinet_depth=1,
    cabinet_width=1,    
):
    """
    Returns (x, y, yaw_degrees)
    """
    r_min, r_max = DRAWER_CONFIGS[drawer_name]["r_min"], DRAWER_CONFIGS[drawer_name]["r_max"]
    r = random.uniform(r_min, r_max)
    print(r)
    theta_rad = np.radians(random.uniform(270-fov_deg/2, 270+fov_deg/2))
    
    x = r * np.cos(theta_rad)
    y = r * np.sin(theta_rad)
    
    
    # Adjust angle b/w drawer origin is bottom right corner of cabinet
    perfect_yaw_rad = np.arctan2(-y, -x)
    yaw_rad = perfect_yaw_rad + np.radians(random.uniform(-face_noise_deg, face_noise_deg))
    
    local_center_x = cabinet_depth / 2.0
    local_center_y = cabinet_width / 2.0
    
    c_yaw = np.cos(yaw_rad)
    s_yaw = np.sin(yaw_rad)
    
    world_offset_x = local_center_x * c_yaw - local_center_y * s_yaw
    world_offset_y = local_center_x * s_yaw + local_center_y * c_yaw
    
    final_origin_x = x - world_offset_x
    final_origin_y = y - world_offset_y
    
    return final_origin_x, final_origin_y, np.degrees(yaw_rad)
    
    
def generate_scenario_string(drawer_name = "label53", **kwargs) -> str:
    drawer_urdf_path = f"{Path.cwd()}/urdf/custom/output/{drawer_name}.urdf"
    
    x, y, yaw = get_random_drawer_pose(drawer_name = drawer_name, **kwargs)
    
    scenario_string = f"""
    directives:
    # add robot
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
        default_joint_positions:
            iiwa_joint_1: [-1.57]
            iiwa_joint_2: [0.1]
            iiwa_joint_3: [0]
            iiwa_joint_4: [-1.2]
            iiwa_joint_5: [0]
            iiwa_joint_6: [1.6]
            iiwa_joint_7: [0]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0

    # add gripper
    - add_model:
        name: wsg
        file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: wsg::body
        X_PC:
            translation: [0, 0, 0.09]
            rotation: !Rpy {{ deg: [90, 0, 90]}}

    # add camera mounted to world 
    - add_frame:
        name: camera0_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [250, 0, 180.0]}}
            translation: [0, 2, 1.4]
    - add_model:
        name: camera0
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: camera0_origin
        child: camera0::base

    # add camera mounted to robot wrist
    - add_frame:
        name: camera_wrist
        X_PF:
            base_frame: iiwa::iiwa_link_7
            translation: [-0.05, 0, 0.07]   
            rotation: !Rpy {{deg: [0, 0, -90]}}
    - add_model:
        name: camera1
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: iiwa::camera_wrist
        child: camera1::base

    # add camera mounted to world pointing top down
    - add_frame:
        name: camera2_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [0, 180.0, 0.0]}}
            translation: [0, -0.5, 3.0]
    - add_model:
        name: camera2
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: camera2_origin
        child: camera2::base
        
    # add random drawer
    - add_model:
        name: drawer
        file: file://{drawer_urdf_path}
    - add_frame:
        name: drawer_origin
        X_PF:
            base_frame: world
            translation: [{x}, {y}, 0]    
            rotation: !Rpy {{ deg: [0, 0, {yaw}]}}  
    - add_weld:
        parent: drawer_origin
        child: drawer::base_link
        
    cameras:
        camera0:
            name: camera0
            depth: True
            X_PB:
                base_frame: camera0::base
        camera1:
            name: camera1
            depth: True
            focal: !FovDegrees
                x: 110   # horizontal FOV in degrees
            X_PB:
                base_frame: camera1::base
        camera2:
            name: camera2
            depth: True
            X_PB:
                base_frame: camera2::base
    
    model_drivers:
        iiwa: !IiwaDriver
            control_mode: position_only 
            hand_model_name: wsg
        wsg: !SchunkWsgDriver {{}}
    """
    return scenario_string

scenario_string = generate_scenario_string(drawer_name = "label55")
scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
diagram = builder.Build()
context = diagram.CreateDefaultContext()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.Initialize()


0.318210655695157


material [ 'None_NONE.006' ] not found in .mtl



### Teleop with XBOX


In [5]:
pygame.init()  # Initialize all Pygame modules
pygame.joystick.init() # Initialize the joystick module

joystick_count = pygame.joystick.get_count()
if joystick_count == 0:
    print("No joysticks found.")
else:
    # Initialize the first detected joystick (assuming one Xbox controller)
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print(f"Controller detected: {joystick.get_name()}")
    
# --- Configuration ---
# Dead Zone threshold for stick inputs
DEAD_ZONE = 0.15 
POLLING_RATE_HZ = 60 

AXIS_LEFT_X  = 0
AXIS_LEFT_Z  = 1
AXIS_RIGHT_X = 2
AXIS_RIGHT_Z = 3
AXIS_LEFT_TRIGGER  = 4
AXIS_RIGHT_TRIGGER = 5 
BUTTON_DPAD_LEFT  = 13
BUTTON_DPAD_RIGHT = 14

def filter_stick_input(value):
    """
    Applies a dead zone filter and scales the remaining input linearly.
    """
    if abs(value) < DEAD_ZONE:
        return 0.0
    
    scaled_value = (abs(value) - DEAD_ZONE) / (1.0 - DEAD_ZONE)
    return math.copysign(scaled_value, value)

Controller detected: Xbox 360 Controller


In [6]:
DRAWER_CONFIGS = {
    "label53": {"handle_label": "handle5", "r_min": 0.28, "r_max": 0.31, "lambda": 0.00004, "kp_null": 1},
    "label62": {"handle_label": "handle3", "r_min": 0.4, "r_max": 0.45, "lambda": 0.004, "kp_null": 5},
    "label31": {"handle_label": "handle3", "r_min": 0.39, "r_max": 0.43,"lambda": 0.0004, "kp_null": 1.5},
    "label55": {"handle_label": "handle7", "r_min": 0.3, "r_max": 0.32,"lambda": 0.00001, "kp_null": 1}

}


In [7]:
class TeleopController(LeafSystem):
    """
    Buttons for end-effector pose to joint positions via ik
    """
    def __init__(self, meshcat, plant):
        LeafSystem.__init__(self)
        
        # Movement params
        self.STEP_SIZE_M = 0.005
        self.STEP_SIZE_RAD = 0.087 # ~5 degrees
        
        # --- PYGAME & CONTROLLER INITIALIZATION ---
        try:
            pygame.init()
            pygame.joystick.init()
            if pygame.joystick.get_count() == 0:
                print("ERROR: No joystick/controller detected! Teleop will be disabled.")
                self.controller = None
            else:
                self.controller = pygame.joystick.Joystick(0)
                self.controller.init()
                print(f"Controller Connected: {self.controller.get_name()}")
        except Exception as e:
            print(f"ERROR: Pygame initialization failed: {e}")
            self.controller = None
            
        self.meshcat = meshcat
        self.plant = plant
        
        # getting the necessary frames
        self.plant_context = plant.CreateDefaultContext()
        self.W_frame = plant.world_frame()

        wsg_instance = self.plant.GetModelInstanceByName("wsg")
        wsg_link = self.plant.GetBodyByName("body", wsg_instance)
        self.WG_frame = wsg_link.body_frame()
        
        self.nq = 7 
        
        # Configuration for IK
        self.q_initial = np.array([-1.57, 0.2, 0, -1.2, 0, 1.6, 0])
        self.iiwa = self.plant.GetModelInstanceByName("iiwa")
        first_iiwa_joint = self.plant.GetJointByName("iiwa_joint_1", self.iiwa)

        self.iiwa_q_start_index = first_iiwa_joint.position_start()
        
        # Initialize positions
        plant.SetPositions(self.plant_context, self.iiwa, self.q_initial)
        
        # initialize blank desired states
        self.X_WG_desired = plant.CalcRelativeTransform(
            self.plant_context, self.W_frame, self.WG_frame
        )
        self.R_WG_desired = self.X_WG_desired.rotation()
        self.q_desired = self.q_initial
        self.wsg_position_desired = 0.1
        
        # Register output ports for IIWA and WSG commands
        self.DeclareVectorOutputPort(
            "iiwa_position_command", 
            BasicVector(self.nq), 
            self.DoCalcIiwaOutput
        )
        self.DeclareVectorOutputPort(
            "wsg_cmd", 
            BasicVector(1), 
            lambda context, output: output.SetFromVector([self.wsg_position_desired])
        )

        meshcat.AddButton("Stop Simulation") 
        
    def read_controller_inputs(self):
        """
        Reads the current state of the Xbox controller and returns normalized 
        XYZ/RPY values.
        """
        if not self.controller:
            return None, None
            
        pygame.event.pump()
        
        # Read raw stick/trigger inputs
        raw_lx = self.controller.get_axis(AXIS_LEFT_X)
        raw_lz = self.controller.get_axis(AXIS_LEFT_Z)
        raw_rx = self.controller.get_axis(AXIS_RIGHT_X)
        raw_rz = self.controller.get_axis(AXIS_RIGHT_Z)

        raw_lt = self.controller.get_axis(AXIS_LEFT_TRIGGER)
        raw_rt = self.controller.get_axis(AXIS_RIGHT_TRIGGER)
        
        def deadzone(val):
            return 0 if abs(val) < 0.1 else val

        
        Y = filter_stick_input(raw_lx)
        Z = filter_stick_input(raw_lz) 
        X = raw_rt - raw_lt 
        
        btn_gripper_close = self.controller.get_button(0) # A button
        btn_gripper_open  = self.controller.get_button(1) # B button

        # RPY rotation
        btn_left  = self.controller.get_button(BUTTON_DPAD_LEFT)
        btn_right = self.controller.get_button(BUTTON_DPAD_RIGHT)
        Pitch = float(btn_right - btn_left)
        Roll = filter_stick_input(raw_rz) 
        Yaw = filter_stick_input(raw_rx)
        
        translation_vector = np.array([X, Y, Z])
        rotation_vector = np.array([Roll, Pitch, Yaw])
        
        return translation_vector, rotation_vector, btn_gripper_close, btn_gripper_open
    
    def step_control(self):
        """
        checks for update events
        """
    
        if not self.controller:
            return

        translation_input, rotation_input, btn_a, btn_b = self.read_controller_inputs()
        delta_p = translation_input * self.STEP_SIZE_M
        
        moved_p = np.linalg.norm(delta_p) > 1e-4
        if moved_p:
            self.X_WG_desired.set_translation(
                self.X_WG_desired.translation() + delta_p
            )
            
        # roll pitch. yaw
        
        roll_rate, pitch_rate, yaw_rate = rotation_input * self.STEP_SIZE_RAD
        delta_R = RotationMatrix.MakeXRotation(roll_rate) \
                  .multiply(RotationMatrix.MakeYRotation(pitch_rate)) \
                  .multiply(RotationMatrix.MakeZRotation(yaw_rate))
                  
        moved_R = np.linalg.norm(rotation_input) > 1e-4 
        
        if moved_R:
            self.R_WG_desired = delta_R.multiply(self.R_WG_desired)

        if btn_a == 1: 
            self.wsg_position_desired = 0.0
        elif btn_b == 1:
            self.wsg_position_desired = 0.1
            
        # recalculate IK if any change occurred
        if moved_p or moved_R or btn_a == 1 or btn_b == 1:
            self.X_WG_desired = RigidTransform(self.R_WG_desired, self.X_WG_desired.translation())
            self._calculate_ik()

            
  
    def DoCalcIiwaOutput(self, context, output):
    #     """
    #     Outputs the latest calculated joint position command.
    #     """
        
        output.SetFromVector(self.q_desired)
        
    def _calculate_ik(self):
        """
        Finds the joint positions (q_desired) that best match the desired
        end-effector pose (X_WG_desired), using the last valid solution as a seed.
        """
        print("finding IK solution...")
        ik = InverseKinematics(self.plant, self.plant_context)
        
        # define decision variables for only the IIWA joints
        q_iiwa = ik.q()[self.iiwa_q_start_index : self.iiwa_q_start_index + self.nq]
        
        p_WG_W = self.X_WG_desired.translation()
        ik.AddPositionConstraint(
            self.WG_frame, [0, 0, 0], self.W_frame, p_WG_W, p_WG_W
        )
        
        R_WG_W = self.X_WG_desired.rotation()
        # ik.AddOrientationConstraint(
        #     self.WG_frame, R_WG_W, self.W_frame, RotationMatrix(), 0.05
        # )
        ik.AddOrientationConstraint(
            self.WG_frame, RotationMatrix(), self.W_frame, R_WG_W, 0.05
        )
        
        prog = ik.prog()
        q_current = self.q_desired
        
        # cost minimizes distance b/w current and future state
        prog.AddQuadraticErrorCost(
            np.identity(self.nq), 
            q_current, 
            q_iiwa
        )
        
        solver = SnoptSolver()
               # set initial guess
        q_full_initial_guess = self.plant.GetPositions(self.plant_context)
        
        result = solver.Solve(prog, initial_guess=q_full_initial_guess)

        if result.is_success():
            # update positions
            q_full_result = result.GetSolution(ik.q())
            q_iiwa_result = q_full_result[self.iiwa_q_start_index : self.iiwa_q_start_index + self.nq]
            
            self.q_desired = q_iiwa_result
            self.plant.SetPositions(self.plant_context, self.iiwa, self.q_desired)
        else:
            print(f"IK Warning: Failed to find joint solution. Check if target is reachable.")


In [8]:
WSG_MIN = 0.0        # fully closed width (meters)
WSG_MAX = 0.1        # fully open width (meters)

def normalize_wsg(width):
    return np.clip((width - WSG_MIN) / (WSG_MAX - WSG_MIN), 0.0, 1.0)

def unnormalize_wsg(norm_width):
    return WSG_MIN + norm_width * (WSG_MAX - WSG_MIN)

# TODO: figure out real velocity limit in simulation by closing/opening grippers
WSG_MAX_SPEED = 0.4  # meters/sec width change

def normalize_wsg_velocity(width_velocity):
    v = width_velocity / WSG_MAX_SPEED
    return np.clip(v, -1.0, 1.0)

Run simulation to collect teleoperation data and save it to files.

In [9]:
def start_simulation(meshcat):
    target_dt = 0.05  # 20 Hz

    scenario = LoadScenario(data=scenario_string)
    station = MakeHardwareStation(scenario, meshcat=meshcat)
    builder = DiagramBuilder()
    builder.AddSystem(station)
    plant = station.GetSubsystemByName("plant") 

    camera_world = station.GetSubsystemByName("rgbd_sensor_camera0")
    camera_wrist = station.GetSubsystemByName("rgbd_sensor_camera1")
    camera_top = station.GetSubsystemByName("rgbd_sensor_camera2")

    controller = builder.AddSystem(TeleopController(meshcat, plant))

    # add controllers
    
    builder.Connect(
        controller.GetOutputPort("iiwa_position_command"),
        station.GetInputPort("iiwa.position")
    )
    builder.Connect(
        controller.GetOutputPort("wsg_cmd"),
        station.GetInputPort("wsg.position")
    )


    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    station_context = diagram.GetSubsystemContext(station, context)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)


    print("\n--- Starting Cartesian Teleop Demo ---")
    print("Please focus the Meshcat viewer window to enable keyboard input.")


    image_data_camera0 = []
    image_data_camera1 = []
    image_data_camera2 = []
    timestamps = []

    qpos_list = []
    qvel_list = []
    action_list = []  # iiwa + gripper commands


    # Run until the user clicks the "Stop Simulation" button in Meshcat
    # try:
    while meshcat.GetButtonClicks('Stop Simulation') == 0:
        controller.step_control()
        
        simulator.AdvanceTo(simulator.get_context().get_time() + target_dt)

        sim_context = simulator.get_context()
        station_context = diagram.GetSubsystemContext(station, sim_context)

        # ---------- get plant state (qpos, qvel) ----------
        plant_context = station.GetSubsystemContext(plant, station_context)
        
        iiwa_model = plant.GetModelInstanceByName("iiwa")
        wsg_model  = plant.GetModelInstanceByName("wsg")

        # iiwa joints
        q_iiwa = plant.GetPositions(plant_context, iiwa_model)
        v_iiwa = plant.GetVelocities(plant_context, iiwa_model)

        # gripper joints
        q_wsg = plant.GetPositions(plant_context, wsg_model)
        v_wsg = plant.GetVelocities(plant_context, wsg_model)
        wsg_width = normalize_wsg(q_wsg[1] - q_wsg[0])
        wsg_vel = normalize_wsg_velocity(v_wsg[1] - v_wsg[0])

        # concatenate to get exactly (7 + 1,) vectors
        qpos = np.concatenate([q_iiwa, [wsg_width]], axis=0).copy()
        qvel = np.concatenate([v_iiwa, [wsg_vel]], axis=0).copy()

        qpos_list.append(qpos)
        qvel_list.append(qvel)

        # ---------- get current action ----------
        controller_context = diagram.GetSubsystemContext(controller, sim_context)
        
        # normalize gripper commands
        iiwa_cmd = controller.GetOutputPort("iiwa_position_command").Eval(controller_context)
        wsg_cmd = controller.GetOutputPort("wsg_cmd").Eval(controller_context)
        wsg_cmd = [normalize_wsg(x) for x in wsg_cmd]

        # flatten to 1D arrays and concatenate
        iiwa_cmd_np = np.asarray(iiwa_cmd).ravel()
        wsg_cmd_np = np.asarray(wsg_cmd).ravel()
        action = np.concatenate([iiwa_cmd_np, wsg_cmd_np], axis=0)
        action_list.append(action)

        # Grab the current images from the output ports
        img0 = station.GetOutputPort("camera0.rgb_image").Eval(station_context)  # ImageRgba8U object
        img1 = station.GetOutputPort("camera1.rgb_image").Eval(station_context)  # ImageRgba8U object
        img2 = station.GetOutputPort("camera2.rgb_image").Eval(station_context)  # ImageRgba8U object

        # Convert to numpy arrays for easier handling
        np_img0 = np.array(np.copy(img0.data)).reshape(img0.height(), img0.width(), -1)  # RGBA
        np_img1 = np.array(np.copy(img1.data)).reshape(img1.height(), img1.width(), -1)
        np_img2 = np.array(np.copy(img2.data)).reshape(img2.height(), img2.width(), -1)
        
        # keep only RGB channels; ACT expects (H, W, 3) uint8 images
        image_data_camera0.append(np_img0[:, :, :3].copy())
        image_data_camera1.append(np_img1[:, :, :3].copy())
        image_data_camera2.append(np_img2[:, :, :3].copy())

        timestamps.append(sim_context.get_time())
# finally:
    # teleop.spacemouse.close()
    return {
        "qpos": qpos_list,
        "qvel": qvel_list,
        "action": action_list,
        "images": {
            "camera0": image_data_camera0,
            "camera1": image_data_camera1,
            "camera2": image_data_camera2
        },
        "timestamp": timestamps
    }

print("Simulation stopped by user.")

Simulation stopped by user.


In [10]:
def save_trajectory_data(data: dict, episode_idx: int = 0, drawer_name="label53"):
    # ---------- convert buffers to numpy arrays ----------
    qpos_arr = np.stack(data['qpos'], axis=0).astype(np.float32)      # (T, nq)
    qvel_arr = np.stack(data['qvel'], axis=0).astype(np.float32)      # (T, nq)
    action_arr = np.stack(data['action'], axis=0).astype(np.float32)  # (T, act_dim)

    images0_arr = np.stack(data['images']['camera0'], axis=0).astype(np.uint8)  # (T, H, W, 3)
    images1_arr = np.stack(data['images']['camera1'], axis=0).astype(np.uint8)  # (T, H, W, 3)
    images2_arr = np.stack(data['images']['camera2'], axis=0).astype(np.uint8)  # (T, H, W, 3)

    timestamps_arr = np.array(data['timestamp'], dtype=np.float64)

    # ---------- write ACT-style HDF5 episode ----------
    h5_path = os.path.join("teleop_data/sim_open_drawer", f"episode_{episode_idx}_{drawer_name}.hdf5")
    print("Saving episode to ", h5_path)

    with h5py.File(h5_path, "w") as root:
        # mark this as simulation data (ACT uses this flag)
        root.attrs["sim"] = True

        # actions: shape (T, act_dim)
        root.create_dataset("action", data=action_arr, compression="gzip")

    # observations group
        obs_grp = root.create_group("observations")
        obs_grp.create_dataset("qpos", data=qpos_arr, compression="gzip")
        obs_grp.create_dataset("qvel", data=qvel_arr, compression="gzip")

        # optional: store timestamps (not strictly required by ACT but often useful)
        obs_grp.create_dataset("time", data=timestamps_arr)

        # images subgroup, one dataset per camera
        img_grp = obs_grp.create_group("images")
        img_grp.create_dataset("camera0", data=images0_arr, compression="gzip")
        img_grp.create_dataset("camera1", data=images1_arr, compression="gzip")
        img_grp.create_dataset("camera2", data=images2_arr, compression="gzip")

    print(f"Wrote ACT-style episode to {h5_path}")

In [11]:
episode_num = 0
trajectory_list = [None]*10


In [46]:
# num_episodes_per_drawer = 10

train_drawers = ["label31", "label53", "label62"]
eval_drawers = ["label55"]
# for i in range(num_episodes_per_drawer):
#     for drawer in eval_drawers:

# for i in range(num_episodes_per_drawer):
meshcat.Delete()
print(f"Starting episode...")
drawer = 'label53'
# episode_num += 1
# scenario_string = generate_scenario_string(drawer_name = drawer)
trajectory_data = start_simulation(meshcat)
# save_trajectory_data(trajectory_data, episode_idx=i, drawer_name = drawer)
print(f"Episode {episode_num} complete.\n")


material [ 'None_NONE.006' ] not found in .mtl



Starting episode...
Controller Connected: Xbox 360 Controller

--- Starting Cartesian Teleop Demo ---
Please focus the Meshcat viewer window to enable keyboard input.
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
finding IK solution...
findi

In [47]:
trajectory_list[episode_num] = trajectory_data
episode_num +=1
episode_num

8

In [50]:
len(trajectory_list[:-2])

8

In [51]:
for i, trajectory_data in enumerate(trajectory_list[:-2]):
    save_trajectory_data(trajectory_data, episode_idx=i+2, drawer_name = drawer)

Saving episode to  teleop_data/sim_open_drawer/episode_2_label53.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_2_label53.hdf5
Saving episode to  teleop_data/sim_open_drawer/episode_3_label53.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_3_label53.hdf5
Saving episode to  teleop_data/sim_open_drawer/episode_4_label53.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_4_label53.hdf5
Saving episode to  teleop_data/sim_open_drawer/episode_5_label53.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_5_label53.hdf5
Saving episode to  teleop_data/sim_open_drawer/episode_6_label53.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_6_label53.hdf5
Saving episode to  teleop_data/sim_open_drawer/episode_7_label53.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_7_label53.hdf5
Saving episode to  teleop_data/sim_open_drawer/episode_8_label53.hdf5
Wrote ACT-style episode to teleop_data/sim

Replay collected joint and gripper data. (Need to restart kernel, run first three cells to import dependencies and define scenario_string and meshcat, then run this cell.)

In [6]:
joint_data = np.load("downsampled_teleop_joint_data.npz")
joint_t_uniform = joint_data["time"]
joint_q_uniform = joint_data["q"]

gripper_data = np.load("downsampled_teleop_gripper_data.npz")
gripper_t_uniform = gripper_data["time"]
gripper_q_uniform = gripper_data["q"]

scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
plant = station.GetSubsystemByName("plant") 

traj_iiwa = PiecewisePolynomial.FirstOrderHold(joint_t_uniform, joint_q_uniform)
traj_wsg = PiecewisePolynomial.FirstOrderHold(gripper_t_uniform, gripper_q_uniform)

builder = DiagramBuilder()
builder.AddSystem(station)  # your station system already created

# Create trajectory sources
iiwa_source = builder.AddSystem(TrajectorySource(traj_iiwa))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg))

# Connect to the plant
builder.Connect(iiwa_source.get_output_port(0),
                station.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(0),
                station.GetInputPort("wsg.position"))

diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

# Run for the duration of the trajectory
simulator.AdvanceTo(joint_t_uniform[-1])

print("Replay finished!")

Replay finished!


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf4fb7c2-18e7-48b8-93e7-63f0209e538b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>